In [11]:
import pandas as pd
import numpy as np
# Load data from a CSV file
data = pd.read_csv('data_banknote_authentication.csv')

train=data.sample(frac=0.7)
test=data.drop(train.index)

train=train.values
test=test.values

In [12]:
x_train=train[:,0:4]
y_train=train[:,-1]
x_test=test[:,0:4]
y_test=test[:,-1]

In [13]:
def c(i,v):
  s=0
  for j in range(0,len(v)):
    if(v[j]==i):
      s=s+1
  return s
  
def gini_index(regions, classes):
  p = 0
  for i in range(len(regions)):
      s = 0
      r = []
      for k in range(len(regions[i])):
          r.append(regions[i][k][len(regions[i][k])-1])
      n = len(r)
      
      # Check if there are no data points in the region
      if n == 0:
          continue
      
      for j in range(len(classes)):
          l = c(classes[j], r)
          s = s + ((l / n) ** 2)
      p = p + (n * (1 - s))
  return p;
  

In [14]:
def split_at_root_node(feature_index, value, dataset):
  L=[]
  R=[]
  dataset=np.array(dataset)
  for i in range(dataset.shape[0]):
      if dataset[i, feature_index] > value:
          R.append(dataset[i])
      else:
          L.append(dataset[i])
  return L, R


In [15]:
def get_best_split(dataset):
    # Find the different classes present in the dataset
    classes = list(set(row[-1] for row in dataset))

    # Initialize variables for the best split
    best_index, best_value, best_gini_score, best_regions = None, None, float('inf'), None

    # Loop over each input feature index
    for feature_index in range(len(dataset[0]) - 1):
        # Loop over each unique value of the current feature
        for row in dataset:
            value = row[feature_index]
            left, right = split_at_root_node(feature_index, value, dataset)
            
            # Calculate Gini index for this split
            regions = [left, right]
            gini_score = gini_index(regions, classes)

            # Check if this split is better than the current best split
            if gini_score < best_gini_score:
                best_index, best_value, best_gini_score, best_regions = feature_index, value, gini_score, regions

    # Return the best split
    return {'index': best_index, 'value': best_value, 'regions': best_regions}


In [16]:
def leaf_output(region):
    # Check if the region is empty
    if not region:
        return None
    
    # Extract outcomes from the region
    outcomes = [row[-1] for row in region]
    
    # Check if outcomes is empty
    if not outcomes:
        return None
    
    # Return the most common outcome in the region
    return max(set(outcomes), key=outcomes.count)

def split(internal_node, max_depth, min_size, depth):
    left, right = internal_node['regions']
    del internal_node['regions']  # Remove the regions from the node dictionary
    
    # Check if the maximum depth has been reached or if the minimum size condition is met
    if depth >= max_depth or len(left) <= min_size or len(right) <= min_size:
        # Make the node a leaf node
        internal_node['left'] = leaf_output(left)
        internal_node['right'] = leaf_output(right)
        return
    
    # If the conditions are not met, find the best split for the node
    # best_split = get_best_split(internal_node)
    internal_node['left']=get_best_split(left)
    internal_node['right']=get_best_split(right)

    # Recursively split the child nodes
    split(internal_node['left'], max_depth, min_size, depth + 1)
    split(internal_node['right'], max_depth, min_size, depth + 1)
    
def build_tree(train, max_depth, min_size):
    root = get_root_node(train)  # Get the root node by finding the best split for the entire dataset
    split(root, max_depth, min_size, 1)  # Split the root node recursively to grow the tree
    return root

# Helper function to get the best split for the root node
def get_root_node(train):
    # Find the best split for the entire dataset
    best_split = get_best_split(train)
    
    # Create the root node
    root = {'index': best_split['index'], 'value': best_split['value']}
    root['regions'] = best_split['regions']
    
    return root
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [17]:
data=np.array(train)
tree = build_tree(data, 5, 10)

In [34]:
print_tree(tree)

[X1 < 0.301]
 [X2 < 7.503]
  [X1 < -0.467]
   [X3 < 6.716]
    [X1 < -2.299]
     [1.0]
     [1.0]
    [X2 < -4.993]
     [1.0]
     [0.0]
   [X3 < 0.097]
    [X2 < 5.010]
     [1.0]
     [0.0]
    [X4 < 0.723]
     [0.0]
     [1.0]
  [X1 < -5.166]
   [X1 < -5.490]
    [1.0]
    [1.0]
   [X1 < -1.254]
    [X1 < -1.254]
     [0.0]
     [None]
    [X1 < -0.206]
     [0.0]
     [0.0]
 [X3 < -4.388]
  [X1 < 2.392]
   [1.0]
   [0.0]
  [X1 < 1.590]
   [X3 < -2.213]
    [X2 < 3.696]
     [1.0]
     [0.0]
    [X4 < 0.066]
     [0.0]
     [0.0]
   [X1 < 2.018]
    [X3 < -2.958]
     [1.0]
     [0.0]
    [X1 < 3.525]
     [0.0]
     [0.0]


In [33]:
def accuracy(tree, X,y):
    c = 0
    for i in range(len(X)):
        prediction = predict(tree, X[i])  # Exclude the last column (target)
        if prediction == y[i]:  # Access the target column
            c = c + 1
    c = c / len(X)
    return c*100
print("Accuracy at training dataset: ",accuracy(tree,x_train,y_train))
print("Accuracy at test dataset: ",accuracy(tree,x_test,y_test))   

Accuracy at training dataset:  97.5
Accuracy at test dataset:  97.08029197080292
